In [1]:
from datetime import date, timedelta
import calendar as ca
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from imp import reload
import utils_venn; reload(utils_venn); from utils_venn import *;

In [ ]:
PATH_DATA_RAW = '/Users/apple/Desktop/NewA/kaggle/'
df_train = pd.read_csv(PATH_DATA_RAW +
    'train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)
item_nbr_u = df_train[df_train.date>pd.datetime(2017,8,10)].item_nbr.unique()

In [4]:
df_test = pd.read_csv(PATH_DATA_RAW +
    "test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [5]:
items = pd.read_csv(PATH_DATA_RAW +
    "items.csv",
).set_index("item_nbr")

In [6]:
    df_2017 = df_train.loc[df_train.date>=pd.datetime(2016,12,26)]
    del df_train

In [7]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

In [8]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

In [9]:
items = items.reindex(df_2017.index.get_level_values(1))

In [9]:
from sklearn.preprocessing import LabelEncoder
def label_encoding(train,column,le):
    col = train[column]
    le.fit(col)
    train[column] = le.transform(train[column])
    return train,le

In [10]:
le_family = LabelEncoder()
items, le_family = label_encoding(items,'family',le_family)

In [11]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

def get_nearwd(date,b_date):
    date_list = pd.date_range(date-timedelta(140),periods=21,freq='7D').date
    result = date_list[date_list<=b_date][-1]
    return result
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        'family':items.reset_index().family,
        'class':items.reset_index()['class'],
        'perishable':items.reset_index().perishable,
        'item_nbr':promo_2017.reset_index().item_nbr,
        'store_nbr':promo_2017.reset_index().store_nbr,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values,
        "unpromo_16aftsum_2017":(1-get_timespan(promo_2017, t2017+timedelta(16), 16, 16)).iloc[:,1:].sum(axis=1).values, 
    })

    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
        for j in [14,60,140]:
            X["aft_promo_{}{}".format(i,j)] = (promo_2017[
                t2017 + timedelta(days=i)]-1).values.astype(np.uint8)
            X["aft_promo_{}{}".format(i,j)] = X["aft_promo_{}{}".format(i,j)]\
                                        *X['promo_{}_2017'.format(j)]
        if i ==15:
            X["bf_unpromo_{}".format(i)]=0
        else:
            X["bf_unpromo_{}".format(i)] = (1-get_timespan(
                    promo_2017, t2017+timedelta(16), 16-i, 16-i)).iloc[:,1:].sum(
                            axis=1).values / (15-i) * X['promo_{}'.format(i)]

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        #X['mean_12_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values        
        
        date = get_nearwd(t2017+timedelta(i),t2017)
        ahead = (t2017-date).days
        if ahead!=0:
            X['ahead0_{}'.format(i)] = get_timespan(df_2017, date+timedelta(ahead), ahead, ahead).mean(axis=1).values
            X['ahead7_{}'.format(i)] = get_timespan(df_2017, date+timedelta(ahead), ahead+7, ahead+7).mean(axis=1).values
        X["day_1_2017_{}1".format(i)]= get_timespan(df_2017, date, 1, 1).values.ravel()
        X["day_1_2017_{}2".format(i)]= get_timespan(df_2017, date-timedelta(7), 1, 1).values.ravel()
        for m in [3,7,14,30,60,140]:
            X["mean_{}_2017_{}1".format(m,i)]= get_timespan(df_2017, date,m, m).\
                mean(axis=1).values
            X["mean_{}_2017_{}2".format(m,i)]= get_timespan(df_2017, date-timedelta(7),m, m).\
                mean(axis=1).values
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [12]:
# add store features
stores = pd.read_csv(PATH_DATA_RAW + "stores.csv")
le_city = LabelEncoder()
le_state = LabelEncoder()
le_type = LabelEncoder()
le_cluster = LabelEncoder()
stores, le_city = label_encoding(stores, 'city',le_city)
stores, le_state = label_encoding(stores, 'state',le_state)
stores, le_type = label_encoding(stores, 'type',le_type)
stores, le_cluster = label_encoding(stores, 'cluster',le_cluster)

In [13]:
def get_n_days_stat(t2017, n,level):
    tmp_item_city = get_timespan(df_2017, t2017, n, n).reset_index()
    tmp_item_city = pd.merge(tmp_item_city, stores, how='left', on='store_nbr')
    tmp_item_city_mean = tmp_item_city.groupby(['item_nbr',level]).mean().reset_index()
    mean_name = level + '_mean_' + str(n)
    median_name = level + '_median_' + str(n)
    std_name = level + '_std_' + str(n)
    tmp_item_city_mean[mean_name] = tmp_item_city_mean.drop(['item_nbr', 'city', 'store_nbr',
                           'state', 'type', 'cluster'], axis = 1).mean(axis = 1)
    tmp_item_city_mean[median_name] = tmp_item_city_mean.drop(['item_nbr', 'city', 'store_nbr',
                           'state', 'type', 'cluster'], axis = 1).median(axis = 1)
    tmp_item_city_mean[std_name] = tmp_item_city_mean.drop(['item_nbr', 'city', 'store_nbr',
                       'state', 'type', 'cluster'], axis = 1).std(axis = 1)
    tmp_item_city_mean = pd.DataFrame({'item_nbr':tmp_item_city_mean.item_nbr,\
                                       level:tmp_item_city_mean[level],\
                                      mean_name:tmp_item_city_mean[mean_name],\
                                      median_name: tmp_item_city_mean[median_name],\
                                      std_name: tmp_item_city_mean[std_name]})
    return tmp_item_city_mean

In [14]:
def get_n_days_stat_global(t2017, n):
    tmp_item_city = get_timespan(df_2017, t2017, n, n).reset_index()
    tmp_item_city = pd.merge(tmp_item_city, stores, how='left', on='store_nbr')
    tmp_item_city_mean = tmp_item_city.groupby(['item_nbr']).mean().reset_index()
    mean_name = '_mean_' + str(n)
    median_name = '_median_' + str(n)
    std_name = '_std_' + str(n``)
    tmp_item_city_mean[mean_name] = tmp_item_city_mean.drop(['item_nbr', 'city', 'store_nbr',
                           'state', 'type', 'cluster'], axis = 1).mean(axis = 1)
    tmp_item_city_mean[median_name] = tmp_item_city_mean.drop(['item_nbr', 'city', 'store_nbr',
                           'state', 'type', 'cluster'], axis = 1).median(axis = 1)
    tmp_item_city_mean[std_name] = tmp_item_city_mean.drop(['item_nbr', 'city', 'store_nbr',
                       'state', 'type', 'cluster'], axis = 1).std(axis = 1)
    tmp_item_city_mean = pd.DataFrame({'item_nbr':tmp_item_city_mean.item_nbr,\
                                      mean_name:tmp_item_city_mean[mean_name],\
                                      median_name: tmp_item_city_mean[median_name],\
                                      std_name: tmp_item_city_mean[std_name]})
    return tmp_item_city_mean

In [15]:
def achieve_pre_n_days_unique(t2017, n):
    tmp_unique_value = get_timespan(df_2017, t2017, n, n).reset_index()
    tmp_unique_value = tmp_unique_value.drop(['store_nbr', 'item_nbr'], axis = 1)
    return tmp_unique_value.apply(lambda x: len(x.unique()), axis=1)

In [16]:
def achieve_no_sales_rate(t2017, n):
    tmp_unique_value = get_timespan(df_2017, t2017, n, n).reset_index()
    tmp_unique_value = tmp_unique_value.drop(['store_nbr', 'item_nbr'], axis = 1)
    tmp_unique_value[tmp_unique_value == 0] = -1
    tmp_unique_value[tmp_unique_value != -1] = 0
    tmp_unique_value[tmp_unique_value == -1] = 1
    return tmp_unique_value.sum(axis = 1)/n

In [17]:
print("Preparing dataset...")
t2017 = date(2017, 6, 7)
X_l, y_l = [], []
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_tmp = pd.merge(X_tmp, stores, how='left', on='store_nbr')
    
    tmp_item_city_mean = get_n_days_stat(t2017 + delta, 7,'city')
    tmp_item_city_mean_14 = get_n_days_stat(t2017 + delta, 14,'city')
    tmp_item_city_mean_31 = get_n_days_stat(t2017 + delta, 31,'city')
    tmp_item_s_mean = get_n_days_stat_global(t2017 + delta, 7)
    tmp_item_s_mean_14 = get_n_days_stat_global(t2017 + delta, 14)
    tmp_item_s_mean_31 = get_n_days_stat_global(t2017 + delta, 31)
    
    X_tmp = pd.merge(X_tmp, tmp_item_city_mean, how='left', on=['item_nbr','city'])
    X_tmp = pd.merge(X_tmp, tmp_item_city_mean_14, how='left', on=['item_nbr','city'])
    X_tmp = pd.merge(X_tmp, tmp_item_city_mean_31, how='left', on=['item_nbr','city'])
    X_tmp = pd.merge(X_tmp, tmp_item_s_mean, how='left', on=['item_nbr'])
    X_tmp = pd.merge(X_tmp, tmp_item_s_mean_14, how='left', on=['item_nbr'])
    X_tmp = pd.merge(X_tmp, tmp_item_s_mean_31, how='left', on=['item_nbr'])
    
    X_tmp['unique_140'] = achieve_pre_n_days_unique(t2017 + delta, 140)
    
    X_tmp['no_sales_7'] = achieve_no_sales_rate(t2017 + delta, 7)
    X_tmp['no_sales_14'] = achieve_no_sales_rate(t2017 + delta, 14)
    X_tmp['no_sales_30'] = achieve_no_sales_rate(t2017 + delta, 30)
    X_tmp['no_sales_60'] = achieve_no_sales_rate(t2017 + delta, 60)
    X_tmp['no_sales_140'] = achieve_no_sales_rate(t2017 + delta, 140)
    
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_val = pd.merge(X_val, stores, how='left', on='store_nbr')
tmp_item_city_mean = get_n_days_stat(date(2017, 7, 26), 7,'city')
tmp_item_city_mean_14 = get_n_days_stat(date(2017, 7, 26), 14,'city')
tmp_item_city_mean_31 = get_n_days_stat(date(2017, 7, 26), 31,'city')
tmp_item_s_mean = get_n_days_stat_global(date(2017, 7, 26), 7)
tmp_item_s_mean_14 = get_n_days_stat_global(date(2017, 7, 26), 14)
tmp_item_s_mean_31 = get_n_days_stat_global(date(2017, 7, 26), 31)

X_val = pd.merge(X_val, tmp_item_city_mean, how='left', on=['item_nbr','city'])
X_val = pd.merge(X_val, tmp_item_city_mean_14, how='left', on=['item_nbr','city'])
X_val = pd.merge(X_val, tmp_item_city_mean_31, how='left', on=['item_nbr','city'])
X_val = pd.merge(X_val, tmp_item_s_mean, how='left', on=['item_nbr'])
X_val = pd.merge(X_val, tmp_item_s_mean_14, how='left', on=['item_nbr'])
X_val = pd.merge(X_val, tmp_item_s_mean_31, how='left', on=['item_nbr'])

X_val['unique_140'] = achieve_pre_n_days_unique(date(2017, 7, 26), 140)

X_val['no_sales_7'] = achieve_no_sales_rate(date(2017, 7, 26), 7)
X_val['no_sales_14'] = achieve_no_sales_rate(date(2017, 7, 26), 14)
X_val['no_sales_30'] = achieve_no_sales_rate(date(2017, 7, 26), 30)
X_val['no_sales_60'] = achieve_no_sales_rate(date(2017, 7, 26), 60)
X_val['no_sales_140'] = achieve_no_sales_rate(date(2017, 7, 26), 140)

X_test = prepare_dataset(date(2017, 8, 16), is_train=False)
X_test = pd.merge(X_test, stores, how='left', on='store_nbr')
tmp_item_city_mean = get_n_days_stat(date(2017, 8, 16), 7,'city')
tmp_item_city_mean_14 = get_n_days_stat(date(2017, 8, 16), 14,'city')
tmp_item_city_mean_31 = get_n_days_stat(date(2017, 8, 16), 31,'city')
tmp_item_s_mean = get_n_days_stat_global(date(2017, 8, 16), 7)
tmp_item_s_mean_14 = get_n_days_stat_global(date(2017, 8, 16), 14)
tmp_item_s_mean_31 = get_n_days_stat_global(date(2017, 8, 16), 31)

X_test = pd.merge(X_test, tmp_item_city_mean, how='left', on=['item_nbr','city'])
X_test = pd.merge(X_test, tmp_item_city_mean_14, how='left', on=['item_nbr','city'])
X_test = pd.merge(X_test, tmp_item_city_mean_31, how='left', on=['item_nbr','city'])
X_test = pd.merge(X_test, tmp_item_s_mean, how='left', on=['item_nbr'])
X_test = pd.merge(X_test, tmp_item_s_mean_14, how='left', on=['item_nbr'])
X_test = pd.merge(X_test, tmp_item_s_mean_31, how='left', on=['item_nbr'])

X_test['unique_140'] = achieve_pre_n_days_unique(date(2017, 7, 26), 140)

X_test['no_sales_7'] = achieve_no_sales_rate(date(2017, 8, 16), 7)
X_test['no_sales_14'] = achieve_no_sales_rate(date(2017, 8, 16), 14)
X_test['no_sales_30'] = achieve_no_sales_rate(date(2017, 8, 16), 30)
X_test['no_sales_60'] = achieve_no_sales_rate(date(2017, 8, 16), 60)
X_test['no_sales_140'] = achieve_no_sales_rate(date(2017, 8, 16), 140)

Preparing dataset...


In [18]:
# oil = pd.read_csv(PATH_DATA_RAW + "oil.csv")

In [19]:
del tmp_item_city_mean
del tmp_item_city_mean_14
del tmp_item_city_mean_31
del tmp_item_s_mean
del tmp_item_s_mean_14
del tmp_item_s_mean_31

In [20]:
import gc
gc.collect()

588

In [21]:
print("Training and predicting models...")
params = {
    'num_leaves': 127,
    'objective': 'regression',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2_root',
    'lambda_l2': 12, 'lambda_l1': 2
}

MAX_ROUNDS = 4000
val_pred = []
test_pred = []
cate_vars = []

for i in range(16):
    val_results = []
    test_results = []
    for seed in range(5):
        params['seed'] = seed
        print("=" * 50)
        print("Step %d" % (i+1))
        print("=" * 50)
        dtrain = lgb.Dataset(
            X_train, label=y_train[:, i],
            categorical_feature=cate_vars,
            weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
        )
        dval = lgb.Dataset(
            X_val, label=y_val[:, i], reference=dtrain,
            weight=items["perishable"] * 0.25 + 1,
            categorical_feature=cate_vars)
        bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS,
            valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=500
        )
        
        val_result = bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS)
        test_result = bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS)
        
        val_results.append(val_result)
        test_results.append(test_result)
        
    preds_val = val_results[0]
    for i in range(4):
        preds_val += val_results[i+1]
    preds_val = preds_val/5
    
    preds_t = test_results[0]
    for i in range(4):
        preds_t += test_results[i+1]
    preds_t = preds_t/5
        
    val_pred.append(preds_val)
    test_pred.append(preds_t)

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose())**0.5)


Training and predicting models...
Step 1


D:\Anaconda\lib\site-packages\lightgbm\basic.py:1005: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.52626	valid_1's rmse: 0.532773
[1000]	training's rmse: 0.515688	valid_1's rmse: 0.53203
Early stopping, best iteration is:
[976]	training's rmse: 0.516139	valid_1's rmse: 0.532019
Step 1
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.526096	valid_1's rmse: 0.532702
[1000]	training's rmse: 0.515564	valid_1's rmse: 0.531943
Early stopping, best iteration is:
[1236]	training's rmse: 0.511298	valid_1's rmse: 0.531836
Step 1
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.526133	valid_1's rmse: 0.532673
[1000]	training's rmse: 0.515605	valid_1's rmse: 0.531979
Early stopping, best iteration is:
[1415]	training's rmse: 0.508284	valid_1's rmse: 0.531773
Step 1
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.526164	valid_1's rmse: 0.532701
[1000]	training's rmse: 0.515599	valid_1's rmse: 0.53

Step 5
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.568521	valid_1's rmse: 0.585485
[1000]	training's rmse: 0.555233	valid_1's rmse: 0.584258
[1500]	training's rmse: 0.545057	valid_1's rmse: 0.583928
Early stopping, best iteration is:
[1478]	training's rmse: 0.545472	valid_1's rmse: 0.583918
Step 5
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.568685	valid_1's rmse: 0.585338
[1000]	training's rmse: 0.555288	valid_1's rmse: 0.584046
[1500]	training's rmse: 0.545018	valid_1's rmse: 0.583655
Early stopping, best iteration is:
[1556]	training's rmse: 0.543979	valid_1's rmse: 0.583638
Step 5
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.568632	valid_1's rmse: 0.585569
[1000]	training's rmse: 0.555181	valid_1's rmse: 0.584308
Early stopping, best iteration is:
[1424]	training's rmse: 0.546504	valid_1's rmse: 0.583907
Step 6
Training until validation scores don't improve

Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.559747	valid_1's rmse: 0.59703
[1000]	training's rmse: 0.547196	valid_1's rmse: 0.59619
Early stopping, best iteration is:
[1007]	training's rmse: 0.547051	valid_1's rmse: 0.596184
Step 10
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.559595	valid_1's rmse: 0.597346
Early stopping, best iteration is:
[948]	training's rmse: 0.54824	valid_1's rmse: 0.596677
Step 10
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.559746	valid_1's rmse: 0.597066
[1000]	training's rmse: 0.547198	valid_1's rmse: 0.596266
Early stopping, best iteration is:
[1293]	training's rmse: 0.541246	valid_1's rmse: 0.596091
Step 10
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.559723	valid_1's rmse: 0.597261
[1000]	training's rmse: 0.547085	valid_1's rmse: 0.596304
Early stopping, best iteration is:
[983]	training's

[500]	training's rmse: 0.558894	valid_1's rmse: 0.581979
Early stopping, best iteration is:
[688]	training's rmse: 0.553538	valid_1's rmse: 0.581689
Step 15
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.55883	valid_1's rmse: 0.581744
Early stopping, best iteration is:
[888]	training's rmse: 0.548719	valid_1's rmse: 0.581286
Step 15
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.558964	valid_1's rmse: 0.581757
Early stopping, best iteration is:
[865]	training's rmse: 0.549265	valid_1's rmse: 0.581295
Step 15
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.558975	valid_1's rmse: 0.581706
[1000]	training's rmse: 0.546147	valid_1's rmse: 0.581222
Early stopping, best iteration is:
[1123]	training's rmse: 0.543523	valid_1's rmse: 0.581163
Step 15
Training until validation scores don't improve for 50 rounds.
[500]	training's rmse: 0.558872	valid_1's rmse: 0.581816
[1000]	tr

In [ ]:
Validation mse: 0.574899765388

Validation mse: 0.572784694105
    
Validation mse: 0.59813112207
    
Validation mse: 0.596577072489 -510
Validation mse: 0.596107717871
Validation mse: 0.595836970438
Validation mse: 0.59489331421
Validation mse: 0.594529921257 = 509
Validation mse: 0.593510745809

In [ ]:
[100]	training's rmse: 0.537131	valid_1's rmse: 0.53378
[200]	training's rmse: 0.532828	valid_1's rmse: 0.529438
[300]	training's rmse: 0.529794	valid_1's rmse: 0.526397
[400]	training's rmse: 0.527279	valid_1's rmse: 0.523862
[500]	training's rmse: 0.524915	valid_1's rmse: 0.521508

In [22]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0).reset_index()
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 10000)
submission.loc[~submission.item_nbr.isin(item_nbr_u),'unit_sales']=0
del item_nbr_u
submission[['id','unit_sales']].to_csv(PATH_DATA_SUBMIT + 'lgb_leave127_bagging.csv', float_format='%.4f', index=None)

Making submission...
